In [1]:
%reload_ext autoreload
%autoreload 2

import numpy as np

import torch
import torchvision as tv

DBG_SEED = 2718
np.random.seed(DBG_SEED)
torch.cuda.manual_seed_all(DBG_SEED)

print("Is cuda working: " + str(torch.cuda.is_available()))

import noiseprofile as noise
import IntelSceneDataset as isds
import classifiers
from classifiers import SimpleClassifier

import denoise

from tqdm.auto import tqdm

Is cuda working: True


In [2]:
noises = {"awn": noise.AdditiveWhiteNoise(0, 0.1),
          "srn": noise.ScalarRayleighNoise(1.0 / np.sqrt(2.0 * np.log(2.0))),
          "dropout": noise.DropoutNoise(0.05)
}

In [3]:
transform = tv.transforms.Compose(
    [tv.transforms.CenterCrop(150),
     tv.transforms.ToTensor(),
     tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [4]:
dbpath = "/mnt/hd-storage/IntelImageClassification"
batchsize = 8

cleanTr = isds.IntelSceneDataset(dbpath, transform=transform)
dataloader = torch.utils.data.DataLoader(cleanTr,
                                           batch_size=batchsize,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=0,
                                           drop_last=True)

In [5]:
batchdims = denoise.CSC.batchdims(cleanTr.shape[2:], cleanTr.shape[1], batchsize)
batchkeys = {"sigshape", "channels", "batchsize"}
dictfname = "test_dict.npy"
csc = denoise.CSC(dictfname, batchdims, dimN=2, dtype=torch.float32)

{'C': 3,
 'Cd': 3,
 'K': 8,
 'M': 240,
 'N': 22500,
 'Nv': (150, 150),
 'axisC': 2,
 'axisK': 3,
 'axisM': 4,
 'axisN': (0, 1),
 'dimC': 1,
 'dimCd': 1,
 'dimK': 1,
 'dimN': 2,
 'shpD': (10, 10, 3, 1, 240),
 'shpS': (150, 150, 3, 8, 1),
 'shpX': (150, 150, 1, 8, 240)}


In [6]:
solveopt = {
    "normed": False
}
restransform = tv.transforms.Compose(
    [tv.transforms.CenterCrop(150),
     tv.transforms.ToTensor(),
#      tv.transforms.Lambda(lambda X: csc.solve(X, **solveopt)),
     tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [7]:
# https://towardsdatascience.com/how-to-train-an-image-classifier-in-pytorch-and-use-it-to-perform-basic-inference-on-single-images-99465a1e9bf5
model = tv.models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False
    
model.fc = torch.nn.Sequential(torch.nn.Linear(2048, 512),
                               torch.nn.ReLU(),
#                                torch.nn.Dropout(0.2),
                               torch.nn.Linear(512, len(cleanTr.categories)),
                               torch.nn.LogSoftmax(dim=1))
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.003)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
epochs = 20
window = 50
for epoch in tqdm(range(epochs), desc="Epoch"):
    cumloss = 0.0
    t = tqdm(enumerate(dataloader), leave=False, total=len(dataloader))
    for m,dataiter in t:
        data = dataiter["images"].to(device)
        labels = dataiter["labels"].to(device)
        
        optimizer.zero_grad()
        logps = model.forward(data)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        
        cumloss += loss.item()
        
        if (m + 1) % window == 0:
            t.set_description(f"Windowed Loss = {cumloss / window:.3f}")
            cumloss=0.0
torch.save(model, 'resnetIS.pt')

In [9]:
cleanTe = isds.IntelSceneDataset(dbpath, segment="test", transform=transform)
tedataloader = torch.utils.data.DataLoader(cleanTe,
                                           batch_size=batchsize,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=0,
                                           drop_last=True)

In [10]:
def predict_image(image):
    image_tensor = test_transforms(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    inp = Variable(image_tensor).to(device)
    out = model(inp)
    index = out.data.cpu().numpy().argmax()
    return index

In [18]:
t = tqdm(enumerate(tedataloader), leave=False, total=len(tedataloader))
C = np.zeros((len(cleanTe.categories), len(cleanTe.categories)), dtype=np.uint)
for m,dataiter in t:
    imgs = dataiter["images"]
    ytrue = dataiter["labels"]
    y = model(imgs.to(device))
    ypred = torch.argmax(model(imgs.to(device)), axis=1).cpu()
    for m in range(ypred.shape[0]):
        C[ytrue[m], ypred[m]] += np.uint(1)
    
    if (m + 1) % 10 == 0:
        accuracy = np.sum(np.diag(C)) / np.sum(C)

        tdesc = f"[{m+1}] Accuracy: {accuracy:.4f}"
        t.set_description(tdesc)
print(C)

[[459   0   3   4   5   3]
 [  4 318  12   7  95   1]
 [  0   0 478  15   4  13]
 [  1   3  33 401   7  80]
 [  5  11  14   2 467   2]
 [  5   0  33  77   8 430]]
